Merging predicted model odds with betfiar historic odds

In [122]:
import pandas as pd
import numpy as np
from scipy import stats

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Load model preds

In [123]:
model_preds = pd.read_csv(r'/total-points-score-model/data/model-predictions/xgb_model_preds.csv')
model_preds = model_preds.drop(columns = ['Date'])
model_preds.head()

,Match_ID,Total_Game_Score,xgb_preds,std_dev
0,200701_Adelaide_Essendon,179,169.72743,32.125702
1,200701_BrisbaneLions_Hawthorn,113,169.72743,32.125702
2,200701_Carlton_Richmond,213,169.72743,32.125702
3,200701_Fremantle_PortAdelaide,226,169.72743,32.125702
4,200701_Melbourne_StKilda,155,169.72743,32.125702


Load aus betting historic odds

In [124]:
betting_odds = pd.read_csv(r'/total-points-score-model/data/aus_sports_betting_total_score_odds.csv')

In [125]:
betting_odds.head()

,Match_ID,Round_ID,Season,Home_Team,Away_Team,Market,Total_Score_Open,Total_Score_Close,Total_Score_Open_Odds,Total_Score_Close_Odds,Total_Score_Close_Implied%,Total_Score_Open_Implied%,Total_Score_Open_Total%,Total_Score_Close_Total%,Total_Score_Open_Margin,Total_Score_Close_Margin,Total_Score_Open_True%,Total_Score_Close_True%,Total_Score_Open_True_Odds,Total_Score_Close_True_Odds
0,201901_Adelaide_Hawthorn,201901,2019,Adelaide,Hawthorn,Over,NaN,185.5,NaN,1.91,0.52356,NaN,0.0,1.04712,0.0,0.04712,NaN,0.5,NaN,2.0
1,201901_Adelaide_Hawthorn,201901,2019,Adelaide,Hawthorn,Under,NaN,185.5,NaN,1.91,0.52356,NaN,0.0,1.04712,0.0,0.04712,NaN,0.5,NaN,2.0
2,201901_BrisbaneLions_WestCoast,201901,2019,Brisbane Lions,West Coast,Over,NaN,186.5,NaN,1.91,0.52356,NaN,0.0,1.04712,0.0,0.04712,NaN,0.5,NaN,2.0
3,201901_BrisbaneLions_WestCoast,201901,2019,Brisbane Lions,West Coast,Under,NaN,186.5,NaN,1.91,0.52356,NaN,0.0,1.04712,0.0,0.04712,NaN,0.5,NaN,2.0
4,201901_Carlton_Richmond,201901,2019,Carlton,Richmond,Over,NaN,188.5,NaN,1.91,0.52356,NaN,0.0,1.04712,0.0,0.04712,NaN,0.5,NaN,2.0


Merge model predictions and betting odds

In [126]:
backtest_data = pd.merge(betting_odds, model_preds, how = 'left', on = "Match_ID")

In [127]:
backtest_data.head()

,Match_ID,Round_ID,Season,Home_Team,Away_Team,Market,Total_Score_Open,Total_Score_Close,Total_Score_Open_Odds,Total_Score_Close_Odds,Total_Score_Close_Implied%,Total_Score_Open_Implied%,Total_Score_Open_Total%,Total_Score_Close_Total%,Total_Score_Open_Margin,Total_Score_Close_Margin,Total_Score_Open_True%,Total_Score_Close_True%,Total_Score_Open_True_Odds,Total_Score_Close_True_Odds,Total_Game_Score,xgb_preds,std_dev
0,201901_Adelaide_Hawthorn,201901,2019,Adelaide,Hawthorn,Over,NaN,185.5,NaN,1.91,0.52356,NaN,0.0,1.04712,0.0,0.04712,NaN,0.5,NaN,2.0,142.0,173.37680,32.125702
1,201901_Adelaide_Hawthorn,201901,2019,Adelaide,Hawthorn,Under,NaN,185.5,NaN,1.91,0.52356,NaN,0.0,1.04712,0.0,0.04712,NaN,0.5,NaN,2.0,142.0,173.37680,32.125702
2,201901_BrisbaneLions_WestCoast,201901,2019,Brisbane Lions,West Coast,Over,NaN,186.5,NaN,1.91,0.52356,NaN,0.0,1.04712,0.0,0.04712,NaN,0.5,NaN,2.0,160.0,159.84569,32.125702
3,201901_BrisbaneLions_WestCoast,201901,2019,Brisbane Lions,West Coast,Under,NaN,186.5,NaN,1.91,0.52356,NaN,0.0,1.04712,0.0,0.04712,NaN,0.5,NaN,2.0,160.0,159.84569,32.125702
4,201901_Carlton_Richmond,201901,2019,Carlton,Richmond,Over,NaN,188.5,NaN,1.91,0.52356,NaN,0.0,1.04712,0.0,0.04712,NaN,0.5,NaN,2.0,161.0,170.74998,32.125702


Calculate Over/Under Predicted Probabilities for the betting Open/Close line

In [128]:
backtest_data['Total_Score_Open_Pred%'] = np.where(backtest_data['Market'] == "Under",
                                                   stats.norm.cdf(backtest_data['Total_Score_Open'],backtest_data['xgb_preds'],backtest_data['std_dev']),
                                                   1 - stats.norm.cdf(backtest_data['Total_Score_Open'],backtest_data['xgb_preds'],backtest_data['std_dev']))

backtest_data['Total_Score_Close_Pred%'] = np.where(backtest_data['Market'] == "Under",
                                                    stats.norm.cdf(backtest_data['Total_Score_Close'],backtest_data['xgb_preds'],backtest_data['std_dev']),
                                                    1 - stats.norm.cdf(backtest_data['Total_Score_Close'],backtest_data['xgb_preds'],backtest_data['std_dev']))

In [129]:
backtest_data.head()

,Match_ID,Round_ID,Season,Home_Team,Away_Team,Market,Total_Score_Open,Total_Score_Close,Total_Score_Open_Odds,Total_Score_Close_Odds,Total_Score_Close_Implied%,Total_Score_Open_Implied%,Total_Score_Open_Total%,Total_Score_Close_Total%,Total_Score_Open_Margin,Total_Score_Close_Margin,Total_Score_Open_True%,Total_Score_Close_True%,Total_Score_Open_True_Odds,Total_Score_Close_True_Odds,Total_Game_Score,xgb_preds,std_dev,Total_Score_Open_Pred%,Total_Score_Close_Pred%
0,201901_Adelaide_Hawthorn,201901,2019,Adelaide,Hawthorn,Over,NaN,185.5,NaN,1.91,0.52356,NaN,0.0,1.04712,0.0,0.04712,NaN,0.5,NaN,2.0,142.0,173.37680,32.125702,NaN,0.352950
1,201901_Adelaide_Hawthorn,201901,2019,Adelaide,Hawthorn,Under,NaN,185.5,NaN,1.91,0.52356,NaN,0.0,1.04712,0.0,0.04712,NaN,0.5,NaN,2.0,142.0,173.37680,32.125702,NaN,0.647050
2,201901_BrisbaneLions_WestCoast,201901,2019,Brisbane Lions,West Coast,Over,NaN,186.5,NaN,1.91,0.52356,NaN,0.0,1.04712,0.0,0.04712,NaN,0.5,NaN,2.0,160.0,159.84569,32.125702,NaN,0.203358
3,201901_BrisbaneLions_WestCoast,201901,2019,Brisbane Lions,West Coast,Under,NaN,186.5,NaN,1.91,0.52356,NaN,0.0,1.04712,0.0,0.04712,NaN,0.5,NaN,2.0,160.0,159.84569,32.125702,NaN,0.796642
4,201901_Carlton_Richmond,201901,2019,Carlton,Richmond,Over,NaN,188.5,NaN,1.91,0.52356,NaN,0.0,1.04712,0.0,0.04712,NaN,0.5,NaN,2.0,161.0,170.74998,32.125702,NaN,0.290297


Convert probabilities to Odds

In [130]:
def convert_probability_to_decimal_odds(prob):
    
    return round(1 / (prob / 1), 2)

In [131]:
backtest_data['Total_Score_Open_Pred_Odds'] = convert_probability_to_decimal_odds(backtest_data['Total_Score_Open_Pred%'])
backtest_data['Total_Score_Close_Pred_Odds'] = convert_probability_to_decimal_odds(backtest_data['Total_Score_Close_Pred%'])

In [132]:
backtest_data.head()

,Match_ID,Round_ID,Season,Home_Team,Away_Team,Market,Total_Score_Open,Total_Score_Close,Total_Score_Open_Odds,Total_Score_Close_Odds,Total_Score_Close_Implied%,Total_Score_Open_Implied%,Total_Score_Open_Total%,Total_Score_Close_Total%,Total_Score_Open_Margin,Total_Score_Close_Margin,Total_Score_Open_True%,Total_Score_Close_True%,Total_Score_Open_True_Odds,Total_Score_Close_True_Odds,Total_Game_Score,xgb_preds,std_dev,Total_Score_Open_Pred%,Total_Score_Close_Pred%,Total_Score_Open_Pred_Odds,Total_Score_Close_Pred_Odds
0,201901_Adelaide_Hawthorn,201901,2019,Adelaide,Hawthorn,Over,NaN,185.5,NaN,1.91,0.52356,NaN,0.0,1.04712,0.0,0.04712,NaN,0.5,NaN,2.0,142.0,173.37680,32.125702,NaN,0.352950,NaN,2.83
1,201901_Adelaide_Hawthorn,201901,2019,Adelaide,Hawthorn,Under,NaN,185.5,NaN,1.91,0.52356,NaN,0.0,1.04712,0.0,0.04712,NaN,0.5,NaN,2.0,142.0,173.37680,32.125702,NaN,0.647050,NaN,1.55
2,201901_BrisbaneLions_WestCoast,201901,2019,Brisbane Lions,West Coast,Over,NaN,186.5,NaN,1.91,0.52356,NaN,0.0,1.04712,0.0,0.04712,NaN,0.5,NaN,2.0,160.0,159.84569,32.125702,NaN,0.203358,NaN,4.92
3,201901_BrisbaneLions_WestCoast,201901,2019,Brisbane Lions,West Coast,Under,NaN,186.5,NaN,1.91,0.52356,NaN,0.0,1.04712,0.0,0.04712,NaN,0.5,NaN,2.0,160.0,159.84569,32.125702,NaN,0.796642,NaN,1.26
4,201901_Carlton_Richmond,201901,2019,Carlton,Richmond,Over,NaN,188.5,NaN,1.91,0.52356,NaN,0.0,1.04712,0.0,0.04712,NaN,0.5,NaN,2.0,161.0,170.74998,32.125702,NaN,0.290297,NaN,3.44


Save to csv

In [133]:
backtest_data.to_csv(r"/total-points-score-model/04-betting-strategy/backtest_data/backtest_data_xgb_preds_2019_2022.csv", index = False)